In [1]:
import re
import difflib
import pandas as pd
from datetime import datetime
from typing import List, Callable, Dict

In [2]:
# Load the customers dataset
customers_df = pd.read_csv(
    "../data/01_raw/olist_customers_dataset.csv",
    converters={
        'customer_zip_code_prefix': str # Convert ZIP codes to strings
    }
)
customers_df

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,03937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,06764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [3]:
# Load the geolocation dataset
geolocation_df = pd.read_csv(
    "../data/01_raw/olist_geolocation_dataset.csv",
    converters={
        'geolocation_zip_code_prefix': str # Convert ZIP codes to strings
    }
)
geolocation_df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [4]:
# Load the orders dataset
orders_df = pd.read_csv("../data/01_raw/olist_orders_dataset.csv")
orders_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [5]:
# Load the products dataset
products_df = pd.read_csv("../data/01_raw/olist_products_dataset.csv")
products_df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [6]:
# Load the sellers dataset
sellers_df = pd.read_csv(
    "../data/01_raw/olist_sellers_dataset.csv",
    converters={
        'seller_zip_code_prefix': str # Convert ZIP codes to strings
    }
)
sellers_df

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,04650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [7]:
def drop_rows_lat_lng(df: pd.DataFrame, lat_lng_conditions: dict) -> pd.DataFrame:
    """
    Removes rows from the DataFrame that do not match the specified conditions for the latitude and longitude.

    Args:
        df (pd.DataFrame): The input DataFrame.
        lat_lng_conditions (dict): A dictionary containing the minimum and maximum lat and lng values.

    Returns:
        pd.DataFrame: The DataFrame with rows matching the conditions removed.
    """
    try:
        # Initialize the mask for rows to keep
        mask = pd.Series(True, index=df.index)
        lat_condition = lat_lng_conditions["geolocation_lat"]
        lng_condition = lat_lng_conditions["geolocation_lng"]

        # Create the conditions
        conditions = {
            'geolocation_lat': lambda x: lat_condition['min'] < x < lat_condition['max'],
            'geolocation_lng': lambda x: lng_condition['min'] < x < lng_condition['max']
        }

        for column, condition in conditions.items():
            # Apply the condition and remove rows which do not meet the condition
            mask &= df[column].apply(condition)
        
        # Return the filtered DataFrame
        return df[mask]
    
    except Exception as e:
        # Showcase the error
        print(f"An unexpected error occurred: {e}")
        return df

In [8]:
conditions = {
    'geolocation_lat': {
        'min': -33.742,
        'max': 5.272
    },
    'geolocation_lng': {
        'min': -73.992,
        'max': -34.792
    }
}

In [9]:
drop_rows_lat_lng(geolocation_df, lat_lng_conditions=conditions)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [10]:
geolocation_df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [11]:
def drop_rows_missing_values(df: pd.DataFrame, subset: List[str], how: str) -> pd.DataFrame:
    """
    Removes rows from the DataFrame that have missing values.

    Args:
        df (pd.DataFrame): The input DataFrame.
        subset (List[str]): The columns to search for missing values.
        how (str): Determines whether to drop rows with any missing or all missing values.

    Returns:
        pd.DataFrame: The DataFrame with the missing values removed from the specified columns.
    """
    try:
        # Ensure the 'how' argument is valid
        if how not in ['any', 'all']:
            raise ValueError("The 'how' parameter must be either 'any' or 'all'.")
        
        # Check if columns exist in the DataFrame
        missing_columns = [col for col in subset if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns not found in the DataFrame: {', '.join(missing_columns)}")

        # Drop rows where the specified columns/fields have null entries
        return df.dropna(subset=subset, how=how)
    
    except ValueError as ve:
        print(f"ValueError: {ve}")
        return df

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return df

In [12]:
products_df.columns.drop('product_id')

Index(['product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [13]:
drop_rows_missing_values(products_df, subset=list(products_df.columns.drop('product_id')), how='all')

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [14]:
products_df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [15]:
def drop_columns(df: pd.DataFrame, columns_to_drop: List[str]) -> pd.DataFrame:
    """
    Drops the specified columns in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns_to_drop (List[str]): List of the specified columns to be removed from the Dataframe.

    Returns:
        pd.DataFrame: The DataFrame with the specified columns removed.
    """
    try:
        # Check if all specified columns exist in the DataFrame
        missing_columns = [col for col in columns_to_drop if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns not found in the DataFrame: {', '.join(missing_columns)}")
        
        # Drop the specified columns from the DataFrame
        return df.drop(columns=columns_to_drop)
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return df 
    
drop_columns(orders_df, columns_to_drop=['order_approved_at', 'order_delivered_carrier_date'])

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-25 23:32:54,2018-02-15 00:00:00


In [16]:
geolocation_df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [17]:
def aggregate_by_column(df: pd.DataFrame, column: str, agg: dict) -> pd.DataFrame:
    """
    Aggregates the dataset by the specified column according to the specified aggregation functions.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The column to group by.
        agg (str): The dictionary to aggregate the other columns by.

    Returns:
        pd.DataFrame: The aggregated DataFrame.
    """
    try:
        # Check if the column exist in the DataFrame
        is_column_missing = True if column not in df.columns else False
        if is_column_missing:
            raise ValueError(f"Column not found in the DataFrame")
        
        # Convert mode to the lambda callable for the aggregation function
        for field, aggregation in agg.items():
            if aggregation == 'mode':
                agg[field] = lambda x: x.mode().iloc[0]
            else:
                agg[field] = aggregation
                
        # Aggregate the input DataFrame by the specified column
        aggregated_df = df.groupby(column).agg(agg)

        # Return the aggregated Dataframe
        return aggregated_df
    
    except ValueError as ve:
        print(f"ValueError: {ve}")
        return df

    except Exception as e:
        # Show the error
        print(f"An unexpected error occurred: {e}")
        return df

agg = {
    'geolocation_lat': 'median',
    'geolocation_lng': 'median',
    'geolocation_city': 'mode',
    'geolocation_state': 'mode',
}


aggregate_by_column(geolocation_df, column='geolocation_zip_code_prefix', agg=agg)

,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
geolocation_zip_code_prefix,,,,
01001,-23.550381,-46.634027,sao paulo,SP
01002,-23.548551,-46.635072,sao paulo,SP
01003,-23.548977,-46.635313,sao paulo,SP
01004,-23.549535,-46.634771,sao paulo,SP
01005,-23.549612,-46.636532,sao paulo,SP
...,...,...,...,...
99960,-27.953797,-52.029641,charrua,RS
99965,-28.179542,-52.035551,agua santa,RS
99970,-28.343273,-51.873734,ciriaco,RS


In [57]:
    
def replace_diacritics(text: str) -> str:
    """
    Replaces diacritics with standard alphabets.

    Args:
        text (str): The input string which may contain diacritic.

    Returns:
        str: Outputs a string where the diacritics is replaced with standard alphabet letters.
    """
    # Initialise the regular expressions for diacritics conversion
    replacements = {
        r'[ãââàáä]': 'a',
        r'[íîì]': 'i',
        r'[úûùü]': 'u',
        r'[éêèë]': 'e',
        r'[óõôòö]': 'o',
        r'[ç]': 'c'
    }

    # Checks if the text is a string
    if isinstance(text, str):
        for pattern, replacement in replacements.items():
            text = re.sub(pattern, replacement, text)
        return text
    else:
        raise ValueError("Text must be a string")


def standardize_cities(df: pd.DataFrame, city_column: str) -> pd.DataFrame:
    """
    Standardizes the names of the cities in the city column of the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        city_column (str): The column containing the city names.

    Returns:
        pd.DataFrame: The DataFrame with standardised names for the cities.
    """
    try:
        # Check if the column exist in the DataFrame
        is_column_missing = True if city_column not in df.columns else False
        if is_column_missing:
            raise ValueError(f"Column not found in the DataFrame")
        
        # Replaces diacritics with plain alphabets
        df[city_column] = df[city_column].apply(replace_diacritics)
        
        return df
    
    except ValueError as ve:
        # Show error
        print(f"ValueError: {ve}")
        return df
    
    except Exception as e:
        # Show error
        print(f"An unexpected error occurred: {e}")
        return df

In [19]:
# Determine the list of citites
customers_df['customer_city'].value_counts()

customer_city
sao paulo                15540
rio de janeiro            6882
belo horizonte            2773
brasilia                  2131
curitiba                  1521
                         ...  
marques de souza             1
general maynard              1
cerrito                      1
mocajuba                     1
governador mangabeira        1
Name: count, Length: 4119, dtype: int64

In [20]:

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


In [21]:
    
# standardize_cities(geolocation_df, city_column='geolocation_city', city_list=''):



In [54]:

def mice_impute_entries(df: pd.DataFrame, imputation_parameters: dict) -> pd.DataFrame:
    """
    Imputes entries in the DataFrame using MICE.
    
    Args:
        df (pd.DataFrame): The dataset with missing entries.
        imputation_parameters (dict): A dictionary that contains the parameters for imputing the missing age entries
            - significant_numerical_columns (List[str]): The numerical columns which are significant for the MICE imputer to fit by.
            - significant_categorical_columns (List[str]): The categorical columns which are significant for the MICE imputer to fit by.
            - max_iter (int, optional): The maximum number of imputation iterations. Default is 10.
            - random_state (int, optional): The seed used by the MICE imputer. Default is 0.
    
    Returns:
        pd.DataFrame: The dataset with imputed entries.
    """
    try:
        # Identify the numerical and categorical columns from the significant columns
        numerical_cols = imputation_parameters.get('numerical_columns', [])
        categorical_cols = imputation_parameters.get('categorical_columns', [])

        # Get the max iteration and random state for the MICE imputer
        max_iter = imputation_parameters.get('max_iter', 10)
        random_state = imputation_parameters.get('random_state', 42)

        # Define a list containing all the significant columns above
        significant_cols = numerical_cols + categorical_cols

        # Check if these columns exist in the DataFrame
        missing_columns = [col for col in significant_cols if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns not found in the DataFrame: {', '.join(missing_columns)}")

        # Identify the columns which are not used in the imputation
        columns_not_used = [col for col in df.columns if col not in significant_cols]

        # Instantiate the IterativeImputer (MICE imputer), OneHotEncoder and StandardScaler
        mice_imputer = IterativeImputer(max_iter=max_iter, 
                                        random_state=random_state,
                                        min_value=0.17,
                                        max_value=80,
                                        initial_strategy='median')
        
        ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        scaler = StandardScaler()

        # Obtain the subset of the dataset
        imputing_df = df[significant_cols]

        # Scale the numerical fields with standard scaling
        if numerical_cols:
            scaled_numerical_data = scaler.fit_transform(imputing_df[numerical_cols])
            scaled_numerical_df = pd.DataFrame(scaled_numerical_data, columns=numerical_cols, index=imputing_df.index)
        else:
            scaled_numerical_df = pd.DataFrame()

        # One hot encode the categorical fields
        if categorical_cols:
            encoded_categorical_data = ohe_encoder.fit_transform(imputing_df[categorical_cols])
            encoded_categorical_df = pd.DataFrame(
                encoded_categorical_data,
                columns=ohe_encoder.get_feature_names_out(categorical_cols),
                index=imputing_df.index
            )
        else:
            encoded_categorical_df = pd.DataFrame()

        # Combine the processed numerical and categorical data
        processed_df = pd.concat([scaled_numerical_df, encoded_categorical_df], axis=1)

        # Apply MICE imputation and convert the data to a dataframe
        imputed_data = mice_imputer.fit_transform(processed_df)
        imputed_df = pd.DataFrame(imputed_data, columns=processed_df.columns, index=processed_df.index)

        # Restore the numerical scaling to the original
        if numerical_cols:
            restored_numerical_data = scaler.inverse_transform(imputed_df[numerical_cols])
            restored_numerical_df = pd.DataFrame(
                restored_numerical_data, columns=numerical_cols, index=imputed_df.index
            )
        else:
            restored_numerical_df = pd.DataFrame()

        # Restore categorical columns to their original categories
        if categorical_cols:
            restored_categorical_data = ohe_encoder.inverse_transform(imputed_df[ohe_encoder.get_feature_names_out(categorical_cols)])
            restored_categorical_df = pd.DataFrame(
                restored_categorical_data, columns=categorical_cols, index=imputed_df.index
            )
        else:
            restored_categorical_df = pd.DataFrame()

        # Combine the restored numerical and categorical columns
        restored_df = pd.concat([restored_numerical_df, restored_categorical_df], axis=1)

        # Combine the restored dataframe with the other columns that were not used
        if columns_not_used:
            restored_df[columns_not_used] = df[columns_not_used]

        return restored_df
    
    except ValueError as ve:
        # Show the error
        print(f"ValueError: {ve}")
        return df
    
    except Exception as e:
        # Show the error
        print(f"An unexpected error occurred: {e}")
        return df

In [52]:
imputation_parameters = {
    'numerical_columns': list(products_df.columns.drop(['product_id', 'product_category_name'])),
    'categorical_columns': [],
    'max_iter': 100,
    'random_state': 42
}

In [53]:
list(products_df.columns.drop(['product_id', 'product_category_name']))

['product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

In [56]:
dataset = mice_impute_entries(products_df, imputation_parameters)
dataset[dataset.isna().any(axis=1)]

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_id,product_category_name


In [26]:
products_df[products_df.isna().any(axis=1)]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,NaN,NaN,NaN,NaN,1800.0,30.0,20.0,70.0
32589,10dbe0fbaa2c505123c17fdc34a63c56,NaN,NaN,NaN,NaN,800.0,30.0,10.0,23.0
32616,bd2ada37b58ae94cc838b9c0569fecd8,NaN,NaN,NaN,NaN,200.0,21.0,8.0,16.0
32772,fa51e914046aab32764c41356b9d4ea4,NaN,NaN,NaN,NaN,1300.0,45.0,16.0,45.0


In [27]:
dataset.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951.000000,32951.00000,32951.000000,32951.000000,32951.000000,32951.000000,32951.000000
mean,48.509987,773.58436,2.194489,2276.517662,30.815252,16.937802,23.196853
std,10.153354,629.40346,1.721082,4281.912701,16.913960,13.637152,12.078691
min,5.000000,4.00000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,344.00000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,604.00000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,961.24668,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.00000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [28]:
products_df.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [29]:
def simple_impute_entries(df: pd.DataFrame, column: str, value: str) -> pd.DataFrame:
    """
    Imputes missing entries in the DataFrame using a specified value.
    
    Args:
        df (pd.DataFrame): The input dataset with missing entries.
        column (str): The column to impute.
        value (str): The value to impute into the empty cell.
    
    Returns:
        pd.DataFrame: The dataset with imputed entries.
    """
    try:
        # Check if the column exist in the DataFrame
        is_column_missing = True if column not in df.columns else False
        if is_column_missing:
            raise ValueError(f"Column not found in the DataFrame")
        
        # Impute the empty cells in the column with the specified value
        df[column] = df[column].fillna(value)
        return df
        
    except ValueError as ve:
        # Show error
        print(f"ValueError: {ve}")
        return df
    
    except Exception as e:
        # Show error
        print(f"An unexpected error occurred: {e}")
        return df

In [30]:
dataset = simple_impute_entries(products_df, column='product_category_name', value='Missing')
dataset[dataset.isna().any(axis=1)]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,Missing,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,Missing,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,Missing,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,Missing,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,Missing,NaN,NaN,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,Missing,NaN,NaN,NaN,1800.0,30.0,20.0,70.0
32589,10dbe0fbaa2c505123c17fdc34a63c56,Missing,NaN,NaN,NaN,800.0,30.0,10.0,23.0
32616,bd2ada37b58ae94cc838b9c0569fecd8,Missing,NaN,NaN,NaN,200.0,21.0,8.0,16.0
32772,fa51e914046aab32764c41356b9d4ea4,Missing,NaN,NaN,NaN,1300.0,45.0,16.0,45.0


In [70]:
def date_difference(row: pd.Series, params: dict) -> int:
    """
    Calculate the difference in days between two dates.

    Parameters:
        row (pd.Series): The row from the dataset.
        mapping (dict): A dictionary containing:
            - 'date1': Column name for the first date.
            - 'date2': Column name for the second date.
            - 'date1_format': Format of the first date.
            - 'date2_format': Format of the second date.

    Returns:
        int: The difference in days between the two dates.
    """
    try:
        # Identify the 2 dates
        date1 = row[params['date1']]
        date2 = row[params['date2']]

        # Check if there are any missing dates
        if pd.isna(date1) or pd.isna(date2):
            return None # Returns None when its not delivered yet
        
        # Parse the dates
        d1 = datetime.strptime(date1, params['date1_format'])
        d2 = datetime.strptime(date2, params['date2_format'])

        # Calculate the difference
        day_difference = (d2 - d1).days
        return day_difference
    
    except ValueError as ve:
        raise ValueError(f"Invalid date or format: {ve}")
    
def feature_engineering(df: pd.DataFrame, new_feature: str, function_name: str, mapping: Dict[str, str]) -> pd.DataFrame:
    """
    Create a new feature based on the features in the row.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        new_feature (str): The column name of the new feature to be created.
        function_name (str): The name of the function to be applied in order to create the feature.
        mapping (List[str]): Mapping column names to the parameter names used by the function. Provides other arguments for the function.

    Returns:
        pd.DataFrame: The DataFrame with new features.
    """
    # Obtain the function based on name
    function = globals()[function_name]

    # Check if the function is valid
    if not callable(function):
        raise ValueError(f"Function is invalid or not callable")
    
    # Initialise the feature engineered DataFrame
    feature_engineered_df = df.copy()
    
    # Apply the function to each row and create the new feature
    feature_engineered_df[new_feature] = df.apply(lambda row: function(row, mapping), axis=1)
    return feature_engineered_df


In [32]:
mapping = {
    'date1': "order_delivered_customer_date",
    'date2': "order_estimated_delivery_date",
    'date1_format': "%Y-%m-%d %H:%M:%S",
    'date2_format': "%Y-%m-%d %H:%M:%S",
}

feature_engineering(orders_df, new_feature='estimated_actual_delivery_offset', function=date_difference, mapping=mapping).sort_values('estimated_actual_delivery_offset')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,estimated_actual_delivery_offset
55619,1b3190b2dfa9d789e1f14c05b647a14a,d306426abe5fca15e54b645e4462dc7b,delivered,2018-02-23 14:57:35,2018-02-23 15:16:14,2018-02-26 18:49:07,2018-09-19 23:24:07,2018-03-15 00:00:00,-189.0
19590,ca07593549f1816d26a572e06dc1eab6,75683a92331068e2d281b11a7866ba44,delivered,2017-02-21 23:31:27,2017-02-23 02:35:15,2017-03-08 13:47:46,2017-09-19 14:36:39,2017-03-22 00:00:00,-182.0
11399,47b40429ed8cce3aee9199792275433f,cb2caaaead400c97350c37a3fc536867,delivered,2018-01-03 09:44:01,2018-01-03 10:31:15,2018-02-06 01:48:28,2018-07-13 20:51:31,2018-01-19 00:00:00,-176.0
81401,2fe324febf907e3ea3f2aa9650869fa5,65b14237885b3972ebec28c0f7dd2220,delivered,2017-03-13 20:17:10,2017-03-13 20:17:10,2017-03-17 07:23:53,2017-09-19 17:00:07,2017-04-05 00:00:00,-168.0
89130,285ab9426d6982034523a855f55a885e,9cf2c3fa2632cee748e1a59ca9d09b21,delivered,2017-03-08 22:47:40,2017-03-08 22:47:40,2017-03-09 08:53:20,2017-09-19 14:00:04,2017-04-06 00:00:00,-167.0
...,...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00,NaN
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaN,NaN,2018-02-06 00:00:00,NaN
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaN,NaN,NaN,2018-09-27 00:00:00,NaN
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaN,NaN,2017-09-15 00:00:00,NaN


In [33]:
customers_df['customer_city'].value_counts().index

Index(['sao paulo', 'rio de janeiro', 'belo horizonte', 'brasilia', 'curitiba',
       'campinas', 'porto alegre', 'salvador', 'guarulhos',
       'sao bernardo do campo',
       ...
       'cambiasca', 'comercinho', 'pires ferreira', 'piquerobi',
       'passo de camaragibe', 'marques de souza', 'general maynard', 'cerrito',
       'mocajuba', 'governador mangabeira'],
      dtype='object', name='customer_city', length=4119)

In [34]:
geolocation_df['geolocation_city'].value_counts().index

Index(['sao paulo', 'rio de janeiro', 'belo horizonte', 'são paulo',
       'curitiba', 'porto alegre', 'salvador', 'guarulhos', 'brasilia',
       'sao bernardo do campo',
       ...
       'ciríaco', 'três arroios', 'mar vermelho', 'poxim', 'olho dágua grande',
       'sao bras', 'são brás', 'ibitiranga', 'quixabá', 'maceia³'],
      dtype='object', name='geolocation_city', length=8011)

In [35]:
dataset = geolocation_df.drop_duplicates().groupby('geolocation_zip_code_prefix').agg({
    'geolocation_city': lambda x: x.mode().iloc[0]
})
dataset

,geolocation_city
geolocation_zip_code_prefix,
01001,sao paulo
01002,sao paulo
01003,sao paulo
01004,sao paulo
01005,sao paulo
...,...
99960,charrua
99965,agua santa
99970,ciriaco


In [36]:
import unicodedata
# Check if the given text contains any diacritic characters.
def identify_diacritics(text):
    for char in text:
        # Normalize the character to NFD and check for combining marks
        if any(unicodedata.category(mark) == 'Mn' for mark in unicodedata.normalize('NFD', char)):
            return True
    return False

# Identify characters in the text that are not letters or diacritics.
def identify_non_letter_and_non_diacritic(text):
    # Initialise the non diacritic characters list
    non_letter_diacritic_chars = []
    
    for char in text:
        # Get the Unicode category of the character
        char_category = unicodedata.category(char)
        # 'L' = Letter categories, 'M' = Mark categories (diacritics)
        if not char_category.startswith(('L', 'M')) and char not in [' ', '-', '\'', '(', ')']:
            non_letter_diacritic_chars.append(char)
    return bool(non_letter_diacritic_chars)

In [37]:
dataset[dataset['geolocation_city'].apply(identify_diacritics)]

,geolocation_city
geolocation_zip_code_prefix,
01133,são paulo
01256,são paulo
01290,são paulo
02170,são paulo
02231,são paulo
...,...
95310,andré da rocha
96130,colônia z-3
97306,são gabriel


In [38]:
dataset[dataset['geolocation_city'].apply(identify_non_letter_and_non_diacritic)]

,geolocation_city
geolocation_zip_code_prefix,
29735,quilometro 14 do mutum
96130,colônia z-3


In [39]:
customers_df[customers_df['customer_zip_code_prefix'] == '29735']

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
6333,94ea98eea015faad90cd64998848d18d,0b7fea4fad79c46f6c10b848e20e9115,29735,quilometro 14 do mutum,ES


In [40]:
set(sellers_df['seller_zip_code_prefix'])

{'48602',
 '04265',
 '76330',
 '04735',
 '89222',
 '72460',
 '36071',
 '14610',
 '86050',
 '21740',
 '24120',
 '21931',
 '14055',
 '03827',
 '38400',
 '03384',
 '59775',
 '88730',
 '93800',
 '12226',
 '30390',
 '02030',
 '12062',
 '17011',
 '89219',
 '72801',
 '03565',
 '09416',
 '85501',
 '09443',
 '12215',
 '12924',
 '05503',
 '12570',
 '37006',
 '06186',
 '20020',
 '72233',
 '18950',
 '01547',
 '19063',
 '01035',
 '38650',
 '15813',
 '22790',
 '01129',
 '23092',
 '05083',
 '05302',
 '13272',
 '29704',
 '18080',
 '98920',
 '44895',
 '04827',
 '02119',
 '58073',
 '85960',
 '13566',
 '14093',
 '35557',
 '17506',
 '03185',
 '35930',
 '15845',
 '15054',
 '09530',
 '04104',
 '13212',
 '03035',
 '08041',
 '03311',
 '06787',
 '03126',
 '03018',
 '87900',
 '11702',
 '13390',
 '89063',
 '33940',
 '15041',
 '13311',
 '83323',
 '89228',
 '36886',
 '06730',
 '19830',
 '01320',
 '75345',
 '78020',
 '14027',
 '36046',
 '09831',
 '03287',
 '38680',
 '95013',
 '87702',
 '90620',
 '20021',
 '17604',


In [41]:
sellers_df[~sellers_df['seller_zip_code_prefix'].isin(set(customers_df['customer_zip_code_prefix']))]

,seller_id,seller_zip_code_prefix,seller_city,seller_state
6,e49c26c3edfa46d227d5121a6b6e4d37,55325,brejao,PE
40,d3674f271c91f824f82d24d92011f669,72233,brasilia,DF
79,78813699ffac347fe27dba345a5f1551,95711,bento goncalves,RS
95,cda598c48d7c614bc1cad7d8ff6f0010,15137,mirassol,SP
129,784ba75dd9d20200c4caed3d7a77141a,01040,sao paulo,SP
...,...,...,...,...
2949,8e29d051f810eb22959ede205b462b9f,15131,mirassol,SP
2982,d66c305afaec317ebee552073a674429,33936,ribeirao das neves,MG
3028,870d0118f7a9d85960f29ad89d5d989a,37708,pocos de caldas,MG
3058,66e0557ecc2b4dbea057e93f215f68d8,04771,sao paulo,SP


In [42]:
sellers_df[~sellers_df['seller_zip_code_prefix'].isin(set(dataset.index)) & ~sellers_df['seller_zip_code_prefix'].isin(set(customers_df['customer_zip_code_prefix']))]

,seller_id,seller_zip_code_prefix,seller_city,seller_state
473,5962468f885ea01a1b6a97a218797b0a,82040,curitiba,PR
791,2aafae69bf4c41fbd94053d9413e87ee,91901,porto alegre,RS
1672,2a50b7ee5aebecc6fd0ff9784a4747d6,72580,brasilia,DF
1931,2e90cb1677d35cfe24eef47d441b7c87,02285,sao paulo,SP
3028,870d0118f7a9d85960f29ad89d5d989a,37708,pocos de caldas,MG


In [43]:
sellers_df[~sellers_df['seller_zip_code_prefix'].isin(set(dataset.index))]

,seller_id,seller_zip_code_prefix,seller_city,seller_state
473,5962468f885ea01a1b6a97a218797b0a,82040,curitiba,PR
791,2aafae69bf4c41fbd94053d9413e87ee,91901,porto alegre,RS
1672,2a50b7ee5aebecc6fd0ff9784a4747d6,72580,brasilia,DF
1931,2e90cb1677d35cfe24eef47d441b7c87,02285,sao paulo,SP
2182,0b3f27369a4d8df98f7eb91077e438ac,07412,aruja,SP
2986,42bde9fef835393bb8a8849cb6b7f245,71551,brasilia,DF
3028,870d0118f7a9d85960f29ad89d5d989a,37708,pocos de caldas,MG


In [44]:
lol = dataset.reset_index()

In [45]:
new = lol.merge(customers_df, how='inner', left_on='geolocation_zip_code_prefix', right_on='customer_zip_code_prefix')
new[~(new['customer_city'] == new['geolocation_city'])].head(50)

,geolocation_zip_code_prefix,geolocation_city,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
665,01256,são paulo,5206bdc4d7ecb9869166f99fc635a5b1,5064f15200f59b32518135b8dfdcf99d,01256,sao paulo,SP
2246,02231,são paulo,5e4b0c76f331945cd0ebb67a3adce403,8d66f98c026e3d77ea0c70f5a2c8e98e,02231,sao paulo,SP
4917,03294,são paulo,7d96202874e65d22c7793fb158ed8495,ad97668eed5826a5d800b15351bea269,03294,sao paulo,SP
10791,04875,são paulo,6fe9ae6b3e2e3ee18fddcfe17f29b226,3fc19a1a15d5d29c01c2f687bd497d00,04875,sao paulo,SP
11676,05120,são paulo,3fcfab0e565a45accff1282416588060,15219300c0299c01fef6cbc7444494e5,05120,sao paulo,SP
11677,05120,são paulo,d7c47b2b00ac1ab4163de1707a3b3ae3,fc8fc2c9ffef45e833ce9734c44aa868,05120,sao paulo,SP
12143,05315,são paulo,ec5f9c9dd3ff056f05411979267887a5,640ac18151727c99757e30b92253dbc0,05315,sao paulo,SP
13182,05606,são paulo,50bcce86319ecdf2df1edeae7272a38e,bd6e919ea010cdfb03825ac482546540,05606,sao paulo,SP
16743,06768,taboão da serra,ceead79d927539feb1a27e3ec80cac36,f75da962fb4ddac1569e90c21803c455,06768,taboao da serra,SP
16744,06768,taboão da serra,d7cf6e6f535ab1be102e3e7ab5185139,659d92dc24d2773bb6c74985a14d2221,06768,taboao da serra,SP


In [46]:
sellers_df

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,04650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [47]:
def cross_reference_cities(
    df: pd.DataFrame, ssot_city_df: pd.DataFrame, mapping: Dict[str, str]
) -> pd.DataFrame:
    """
    Cross-references the input DataFrame with a single source of truth (SSOT) DataFrame 
    containing Brazilian zip codes and cities.

    Args:
        df (pd.DataFrame): The input DataFrame.
        ssot_city_df (pd.DataFrame): The SSOT DataFrame containing Brazilian zip codes and cities.
        mapping (Dict[str, str]): Maps the column name of the zip code and city in the datasets.
            zip_code (str): The column name in df representing the zip code.
            city (str): The column name in df representing the city name.
            true_zip_code (str): The column name in ssot_city_df representing the zip code in ssot_city_df.
            true_city (str): The column name in ssot_city_df representing the true city name.

    Returns:
        pd.DataFrame: The updated DataFrame with consistent city and zip codes.
    """
    # Extract the column mappings
    zip_code = mapping.get('zip_code')
    city = mapping.get('city')
    true_zip_code = mapping.get('true_zip_code')
    true_city = mapping.get('true_city')

    # Initialise an updated DataFrame
    updated_df = df.copy()

    try:
        # Check if required columns exist in the DataFrames
        missing_columns_df = [col for col in [zip_code, city] if col not in df.columns]
        missing_columns_ssot = [col for col in [true_zip_code, true_city] if col not in ssot_city_df.columns]

        if missing_columns_df:
            raise ValueError(f"Columns missing in input DataFrame: {', '.join(missing_columns_df)}")
        if missing_columns_ssot:
            raise ValueError(f"Columns missing in SSOT DataFrame: {', '.join(missing_columns_ssot)}")

        # Create a lookup DataFrame for mode (most frequent) values based on `zip_code`
        ssot_lookup = ssot_city_df.groupby(true_zip_code)[true_city] \
            .agg(lambda x: x.mode()[0] if not x.empty else None)

        # Update inconsistent city and zip codes in the original DataFrame
        updated_df[city] = df[zip_code].apply(lambda x: ssot_lookup.loc[x] if x in ssot_lookup.index else x)

        return updated_df

    except ValueError as ve:
        print(f"ValueError: {ve}")
        return df

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return df

In [48]:
mapsl = {
    'zip_code': 'seller_zip_code_prefix',
    'city': 'seller_city',
    'true_zip_code': 'customer_zip_code_prefix',
    'true_city': 'customer_city'
}

cross_reference_cities(sellers_df, customers_df, mapsl)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi-guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,04650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


In [49]:
customers_df

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,03937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,06764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [50]:
orders_df[(orders_df['order_status'] == 'delivered') & (pd.isna(orders_df['order_delivered_customer_date']))]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
3002,2d1e2d5bf4dc7227b3bfebb81328c15f,ec05a6d8558c6455f0cbbd8a420ad34f,delivered,2017-11-28 17:44:07,2017-11-28 17:56:40,2017-11-30 18:12:23,NaN,2017-12-18 00:00:00
20618,f5dd62b788049ad9fc0526e3ad11a097,5e89028e024b381dc84a13a3570decb4,delivered,2018-06-20 06:58:43,2018-06-20 07:19:05,2018-06-25 08:05:00,NaN,2018-07-16 00:00:00
43834,2ebdfc4f15f23b91474edf87475f108e,29f0540231702fda0cfdee0a310f11aa,delivered,2018-07-01 17:05:11,2018-07-01 17:15:12,2018-07-03 13:57:00,NaN,2018-07-30 00:00:00
79263,e69f75a717d64fc5ecdfae42b2e8e086,cfda40ca8dd0a5d486a9635b611b398a,delivered,2018-07-01 22:05:55,2018-07-01 22:15:14,2018-07-03 13:57:00,NaN,2018-07-30 00:00:00
82868,0d3268bad9b086af767785e3f0fc0133,4f1d63d35fb7c8999853b2699f5c7649,delivered,2018-07-01 21:14:02,2018-07-01 21:29:54,2018-07-03 09:28:00,NaN,2018-07-24 00:00:00
92643,2d858f451373b04fb5c984a1cc2defaf,e08caf668d499a6d643dafd7c5cc498a,delivered,2017-05-25 23:22:43,2017-05-25 23:30:16,NaN,NaN,2017-06-23 00:00:00
97647,ab7c89dc1bf4a1ead9d6ec1ec8968a84,dd1b84a7286eb4524d52af4256c0ba24,delivered,2018-06-08 12:09:39,2018-06-08 12:36:39,2018-06-12 14:10:00,NaN,2018-06-26 00:00:00
98038,20edc82cf5400ce95e1afacc25798b31,28c37425f1127d887d7337f284080a0f,delivered,2018-06-27 16:09:12,2018-06-27 16:29:30,2018-07-03 19:26:00,NaN,2018-07-19 00:00:00


In [105]:
def cross_reference_cities(
    df: pd.DataFrame, ssot_city_df: pd.DataFrame, mapping: Dict[str, str]
) -> pd.DataFrame:
    """
    Cross-references the input DataFrame with a single source of truth (SSOT) DataFrame 
    containing Brazilian zip codes and cities.

    Args:
        df (pd.DataFrame): The input DataFrame.
        ssot_city_df (pd.DataFrame): The SSOT DataFrame containing Brazilian zip codes and cities.
        mapping (Dict[str, str]): Maps the column name of the zip code and city in the datasets.
            zip_code (str): The column name in df representing the zip code.
            city (str): The column name in df representing the city name.
            true_zip_code (str): The column name in ssot_city_df representing the zip code in ssot_city_df.
            true_city (str): The column name in ssot_city_df representing the true city name.

    Returns:
        pd.DataFrame: The updated DataFrame with consistent city and zip codes.
    """
    # Extract the column mappings
    zip_code = mapping.get('zip_code')
    city = mapping.get('city')
    true_zip_code = mapping.get('true_zip_code')
    true_city = mapping.get('true_city')

    # Initialise an updated DataFrame
    updated_df = df.copy()

    try:
        # Check if required columns exist in the DataFrames
        missing_columns_df = [col for col in [zip_code, city] if col not in df.columns]
        missing_columns_ssot = [col for col in [true_zip_code, true_city] if col not in ssot_city_df.columns]

        if missing_columns_df:
            raise ValueError(f"Columns missing in input DataFrame: {', '.join(missing_columns_df)}")
        if missing_columns_ssot:
            raise ValueError(f"Columns missing in SSOT DataFrame: {', '.join(missing_columns_ssot)}")

        # Create a lookup DataFrame for mode (most frequent) values based on `zip_code`
        ssot_lookup = ssot_city_df.groupby(true_zip_code)[true_city] \
            .agg(lambda x: x.mode()[0] if not x.empty else None)

        # Update inconsistent city and zip codes in the original DataFrame
        updated_df[city] = df.apply(lambda row: ssot_lookup.loc[row[zip_code]] if row[zip_code] in ssot_lookup.index else row[city], axis=1)

        return updated_df

    except ValueError as ve:
        print(f"ValueError: {ve}")
        return df

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return df

In [110]:
new_sellers_df = cross_reference_cities(geolocation_df, customers_df, 
                       {
                        'zip_code': 'geolocation_zip_code_prefix',
                        'city': 'geolocation_city',
                        'true_zip_code': 'customer_zip_code_prefix',
                        'true_city': 'customer_city'
                       })

In [107]:
new_sellers_df[new_sellers_df.isna().any(axis=1)]


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state


In [109]:
new_sellers_df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [108]:
new_sellers_df['geolocation_city'].value_counts()

geolocation_city
sao paulo                159919
rio de janeiro            62152
belo horizonte            27806
curitiba                  16594
porto alegre              13522
                          ...  
sao pedro dos crentes         1
sao vicente do grama          1
bicuiba                       1
tres aliancas                 1
ribamar fiquene               1
Name: count, Length: 6418, dtype: int64

In [89]:
geolocation_df['geolocation_city'].value_counts()

geolocation_city
sao paulo         135800
rio de janeiro     62151
belo horizonte     27805
são paulo          24918
curitiba           16593
                   ...  
sao bras               1
são brás               1
ibitiranga             1
quixabá                1
maceia³                1
Name: count, Length: 8011, dtype: int64

In [77]:
def date_difference(row: pd.Series, params: dict) -> int:
    """
    Calculate the difference in days between two dates.

    Parameters:
        row (pd.Series): The row from the dataset.
        mapping (dict): A dictionary containing:
            - 'actual_date': Column name for actual delivery first date.
            - 'estimated_date': Column name for the estimated delivery date.
            - 'actual_date_format': Format of the actual delivery date.
            - 'estimated_date_format': Format of the estimated delivery date.

    Returns:
        int: The difference in days between the two dates.
    """

    print(row)
    try:
        # Identify the 2 dates
        date1 = row[params['actual_date']]
        date2 = row[params['estimated_date']]
        # Check if there are any missing dates
        if pd.isna(date1) or pd.isna(date2):
            return None # Returns None when its not delivered yet
        
        # Parse the dates
        d1 = datetime.strptime(date1, params['actual_date_format'])
        d2 = datetime.strptime(date2, params['estimated_date_format'])

        # Calculate the difference
        day_difference = (d2 - d1).days
        return day_difference
    
    except ValueError as ve:
        raise ValueError(f"Invalid date or format: {ve}")
    
def feature_engineering(df: pd.DataFrame, new_feature: str, function_name: str, mapping: Dict[str, str]) -> pd.DataFrame:
    """
    Create a new feature based on the features in the row.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        new_feature (str): The column name of the new feature to be created.
        function_name (str): The name of the function to be applied in order to create the feature.
        mapping (List[str]): Mapping column names to the parameter names used by the function. Provides other arguments for the function.

    Returns:
        pd.DataFrame: The DataFrame with new features.
    """
    # Obtain the function based on name
    function = globals()[function_name]

    # Check if the function is valid
    if not callable(function):
        raise ValueError(f"Function is invalid or not callable")
    
    # Initialise the feature engineered DataFrame
    feature_engineered_df = df.copy()
    
    # Apply the function to each row and create the new feature
    feature_engineered_df[new_feature] = df.apply(lambda row: function(row, mapping), axis=1)
    return feature_engineered_df


In [76]:
feature_engineering(orders_df, 'new', 'date_difference', {
         'actual_date': 'order_delivered_customer_date',
         'estimated_date': 'order_estimated_delivery_date',
         'actual_date_format': '%Y-%m-%d %H:%M:%S',
         'estimated_date_format': '%Y-%m-%d %H:%M:%S',
    })

order_id                         e481f51cbdc54678b7cc49136f2d6af7
customer_id                      9ef432eb6251297304e76186b10a928d
order_status                                            delivered
order_purchase_timestamp                      2017-10-02 10:56:33
order_approved_at                             2017-10-02 11:07:15
order_delivered_carrier_date                  2017-10-04 19:55:00
order_delivered_customer_date                 2017-10-10 21:25:13
order_estimated_delivery_date                 2017-10-18 00:00:00
Name: 0, dtype: object


KeyError: 'date1'

In [111]:
def drop_rows_lat_lng(df: pd.DataFrame, lat_lng_conditions: dict) -> pd.DataFrame:
    """
    Removes rows from the DataFrame that do not match the specified conditions for the latitude and longitude.

    Args:
        df (pd.DataFrame): The input DataFrame.
        lat_lng_conditions (dict): A dictionary containing the minimum and maximum lat and lng values.

    Returns:
        pd.DataFrame: The DataFrame with rows matching the conditions removed.
    """
    try:
        # Initialize the mask for rows to keep
        mask = pd.Series(True, index=df.index)
        lat_condition = lat_lng_conditions["geolocation_lat"]
        lng_condition = lat_lng_conditions["geolocation_lng"]

        # Create the conditions
        conditions = {
            'geolocation_lat': lambda x: lat_condition['min'] < x < lat_condition['max'],
            'geolocation_lng': lambda x: lng_condition['min'] < x < lng_condition['max']
        }

        for column, condition in conditions.items():
            # Apply the condition and remove rows which do not meet the condition
            mask &= df[column].apply(condition)
        
        # Return the filtered DataFrame
        return df[mask]
    
    except Exception as e:
        # Showcase the error
        print(f"An unexpected error occurred: {e}")
        return df

In [114]:
drop_rows_lat_lng(geolocation_df, {
    'geolocation_lat':{
      'min': -33.742,
      'max': 5.272
    },

    'geolocation_lng':{
      'min': -73.992,
      'max': -34.792
    }
}
).groupby('geolocation_zip_code_prefix')[['geolocation_city']].count()

,geolocation_city
geolocation_zip_code_prefix,
01001,26
01002,13
01003,17
01004,22
01005,25
...,...
99960,5
99965,6
99970,21


In [121]:
def drop_erroneous_orders(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes orders that were considered delivered but do not have a customer delivered date.

    Args:
        df (pd.DataFrame): The input orders DataFrame.

    Returns:
        pd.DataFrame: The DataFrame without the erroneous orders.
    """
    try:
        # Filters out the erroneous orders
        df = df[~((df['order_status'] == 'delivered') & pd.isna(df['order_delivered_customer_date']))]
        return df
    
    except Exception as e:
        print(f"An unexpected error occurred when dropping erroneous orders: {e}")
        return df

In [122]:
drop_erroneous_orders(orders_df)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00
